In [ ]:
!pip install pymupdf pytesseract opencv-python
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

import fitz
import pytesseract
import cv2
import os
from google.colab import files

def pdf_to_images(pdf_path, output_folder):
    pdf = fitz.open(pdf_path)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    page_image_paths = []
    for page_num in range(len(pdf)):
        page = pdf.load_page(page_num)
        pix = page.get_pixmap()
        image_path = f"{output_folder}/page_{page_num+1}.png"
        pix.save(image_path)
        page_image_paths.append(image_path)

    return page_image_paths

def extract_text_from_images(image_paths, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path)
        text = pytesseract.image_to_string(img)
        page_num = os.path.basename(image_path).split('_')[1].split('.')[0]
        text_file_path = os.path.join(output_folder, f'page_{page_num}_text.txt')
        with open(text_file_path, "w") as text_file:
            text_file.write(text)
        print(f"Text extracted from {image_path} and saved to {text_file_path}")
def detect_and_save_images(image_paths, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 50, 150)

        contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        page_num = os.path.basename(image_path).split('_')[1].split('.')[0]
        page_folder = os.path.join(output_folder, f'page_{page_num}')
        os.makedirs(page_folder, exist_ok=True)

        for idx, contour in enumerate(contours):
            x, y, w, h = cv2.boundingRect(contour)
            if w > 50 and h > 50:
                cropped_image = img[y:y+h, x:x+w]
                cropped_image_path = os.path.join(page_folder, f'image_{idx+1}.png')
                cv2.imwrite(cropped_image_path, cropped_image)
                print(f"Image saved at: {cropped_image_path}")
def process_pdf(pdf_path):
    image_output_folder = 'pdf_pages'
    text_output_folder = 'extracted_text'
    image_save_folder = 'extracted_images'
    page_image_paths = pdf_to_images(pdf_path, image_output_folder)
    extract_text_from_images(page_image_paths, text_output_folder)
    detect_and_save_images(page_image_paths, image_save_folder)
    !zip -r extracted_results.zip extracted_text extracted_images
    files.download('extracted_results.zip')
pdf_path = '/content/TTA1.pdf'
process_pdf(pdf_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 87.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.6 MB/s)
debconf: unable

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>